In [1]:
import pandas as pd
import numpy as np
import requests 
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values

# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 
    
# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize
import folium # plotting library
import lxml
print('Folium installed')

Folium installed


In [29]:
pip install geocoder

  Created wheel for future: filename=future-0.18.2-py3-none-any.whl size=491062 sha256=d01feb215e77b5a6a60fa620a309270894fd8c6377f834a43f14277ad7fb0315
  Stored in directory: c:\users\jonas\appdata\local\pip\cache\wheels\56\b0\fe\4410d17b32f1f0c3cf54cdfb2bc04d7b4b8f4ae377e2229ba0
Successfully built future
Note: you may need to restart the kernel to use updated packages.


In [30]:
import geocoder

<h1> This is part one for the final assignment </h1> 

First step: Read the Data form wiki as DataFrame

In [18]:
Wiki_Url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
Wiki = requests.get(Wiki_Url)

wiki_data = pd.read_html(Wiki.content, header = 0)[0]
wiki_data

,Postal Code,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
...,...,...,...
175,M5Z,Not assigned,Not assigned
176,M6Z,Not assigned,Not assigned
177,M7Z,Not assigned,Not assigned
178,M8Z,Etobicoke,"Mimico NW, The Queensway West, South of Bloor,..."


Kill the Neighborhoods that are not assigned

In [19]:
df_Ca = wiki_data[wiki_data.Neighbourhood != 'Not assigned']
df_Ca.reset_index(inplace = True)
df_Ca.head()

,index,Postal Code,Borough,Neighbourhood
0,2,M3A,North York,Parkwoods
1,3,M4A,North York,Victoria Village
2,4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,5,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [20]:
df_Ca = df_Ca.rename(columns = {"Neighbourhood":"Neighborhood"})

I use groupby to goub the neighborhoods with the same postal code

In [21]:
df_Ca.groupby(['Postal Code']).first()

,index,Borough,Neighborhood
Postal Code,,,
M1B,9,Scarborough,"Malvern, Rouge"
M1C,18,Scarborough,"Rouge Hill, Port Union, Highland Creek"
M1E,27,Scarborough,"Guildwood, Morningside, West Hill"
M1G,36,Scarborough,Woburn
M1H,45,Scarborough,Cedarbrae
...,...,...,...
M9N,98,York,Weston
M9P,107,Etobicoke,Westmount
M9R,116,Etobicoke,"Kingsview Village, St. Phillips, Martin Grove ..."


Change the Index Postalcode with the column index

In [25]:
df_Ca = df_Ca.rename(columns = {'Postal Code':'Postal Code'})
df_Ca.set_index("index")

Check whether a Borough is not assigned

In [26]:
df_Ca[df_Ca['Borough'] == 'Not assigned']

,index,Postal Code,Borough,Neighborhood


In [31]:
df_Ca.shape

(103, 4)

<h1> This is part two </h1>

In [34]:
df_la_long = pd.read_csv("https://cocl.us/Geospatial_data")
df_la_long.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [35]:
df_Ca = df_Ca.join(df_la_long.set_index('Postal Code'), on='Postal Code')
df_Ca

,index,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,2,M3A,North York,Parkwoods,43.753259,-79.329656
1,3,M4A,North York,Victoria Village,43.725882,-79.315572
2,4,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,5,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
...,...,...,...,...,...,...
98,160,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North",43.653654,-79.506944
99,165,M4Y,Downtown Toronto,Church and Wellesley,43.665860,-79.383160
100,168,M7Y,East Toronto,"Business reply mail Processing Centre, South C...",43.662744,-79.321558
101,169,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu...",43.636258,-79.498509
